# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from vacation_api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,279.35,92,100,3.89,SJ,1722470817
1,1,port-aux-francais,-49.3500,70.2167,278.00,98,100,12.57,TF,1722470817
2,2,bama,11.5221,13.6856,296.07,95,100,2.52,NG,1722470818
3,3,adamstown,-25.0660,-130.1015,295.43,87,23,9.54,PN,1722470818
4,4,vila franca do campo,37.7167,-25.4333,295.38,83,43,1.54,PT,1722470818


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.query("`Max Temp` <= 299")

# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,279.35,92,100,3.89,SJ,1722470817
1,1,port-aux-francais,-49.3500,70.2167,278.00,98,100,12.57,TF,1722470817
2,2,bama,11.5221,13.6856,296.07,95,100,2.52,NG,1722470818
3,3,adamstown,-25.0660,-130.1015,295.43,87,23,9.54,PN,1722470818
4,4,vila franca do campo,37.7167,-25.4333,295.38,83,43,1.54,PT,1722470818


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()
hotel_df = hotel_df[["City", "Country", "Humidity", "Lat", "Lng"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Humidity,Lat,Lng,Hotel Name
0,olonkinbyen,SJ,92,70.9221,-8.7187,
1,port-aux-francais,TF,98,-49.3500,70.2167,
2,bama,NG,95,11.5221,13.6856,
3,adamstown,PN,87,-25.0660,-130.1015,
4,vila franca do campo,PT,83,37.7167,-25.4333,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
#Testing connection to api and api key validity

target_city = "Seattle"
base_url = "https://api.geoapify.com/v2/places"

# Get the latitude and longitude coordinates of Seattle using a geocoding API
geocoding_url = f"https://api.geoapify.com/v1/geocode/search?text={target_city}&apiKey={geoapify_key}"
geocoding_data = requests.get(geocoding_url).json()

# Extract the latitude and longitude of Seattle
lat = geocoding_data['features'][0]['properties']['lat']
lon = geocoding_data['features'][0]['properties']['lon']

# Make a request to the Geoapify Places API to retrieve hotel accommodations in Seattle
target_url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&lat={lat}&lon={lon}&format=json&apiKey={geoapify_key}"
geo_data = requests.get(target_url).json()

print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -122.33195900000001,
                    47.60375340003019
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "The Arctic Club Seattle",
                "address_line2": "700 3rd Avenue, Seattle, WA 98104, United States of America",
                "brand": "DoubleTree",
                "brand_details": {
                    "wikidata": "Q2504643"
                },
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Seattle",
                "contact": {
                    "phone": "+1-206-340-0340"
                },
                "country": "United States",
                "country_code": "us",
                "county": "King County",
                "datasource": {
                    "attributio

In [19]:
# Set parameters to search for a hotel
limit = 1
radius = 1000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
    "limit": limit,
    "categories": "accommodation.hotel",
    "filter": f"circle:{lng},{lat},{radius}",
    "apiKey": geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response_data = response.json()

    # Print the full API response for debugging
    print(f"API Response for {hotel_df.loc[index, 'City']}: {response_data}")
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
API Response for olonkinbyen: {'type': 'FeatureCollection', 'features': []}
olonkinbyen - nearest hotel: No hotel found
API Response for port-aux-francais: {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Keravel', 'country': 'France', 'country_code': 'fr', 'state': 'French Southern and Antarctic Lands', 'city': 'Port-aux-Français', 'municipality': 'Kerguelen Archipelago', 'street': 'Chemin de Keravel', 'lon': 70.21991619089022, 'lat': -49.34887615, 'formatted': 'Chemin de Keravel, Port-aux-Français, France', 'address_line1': 'Chemin de Keravel', 'address_line2': 'Port-aux-Français, France', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Keravel', 'osm_id': 187212461, 'tour

,City,Country,Humidity,Lat,Lng,Hotel Name
0,olonkinbyen,SJ,92,70.9221,-8.7187,No hotel found
1,port-aux-francais,TF,98,-49.3500,70.2167,Keravel
2,bama,NG,95,11.5221,13.6856,No hotel found
3,adamstown,PN,87,-25.0660,-130.1015,No hotel found
4,vila franca do campo,PT,83,37.7167,-25.4333,Vinha d'Areia Beach Hotel
...,...,...,...,...,...,...
552,myskhako,RU,73,44.6714,37.7433,No hotel found
553,tazacorte,ES,88,28.6290,-17.9293,No hotel found
556,morondava,MG,46,-20.2833,44.2833,Hotel Sharon Menabe
557,pacasmayo,PE,84,-7.4006,-79.5714,Hotel La Estación


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display
# Drop hotels that weren't found
hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == "No hotel found"].index)
hotel_df

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot_2 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)